# GD-MLP

In [ ]:
import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import math
from tqdm import tqdm

from argparse import ArgumentParser

import wandb
from pytorch_lightning.loggers import WandbLogger

wandb.init(project="TSF", name="GD-MLP")
wandb_logger = WandbLogger(project="TSF", name="GD-MLP")

parser = ArgumentParser(description="GD-MLP")

# data loader
parser.add_argument('--data', type=str, default='Exchange', help='dataset type')

# forecasting task
parser.add_argument('--seq_len', type=int, default=336, help='input sequence length')
parser.add_argument('--label_len', type=int, default=336, help='start token length')
parser.add_argument('--pred_len', type=int, default=336, help='prediction sequence length')

# GD-MLP
parser.add_argument('--num_features', type=int, default=1)
parser.add_argument('--hidden_units', type=int, default=512)

# DLinear
# parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# parser.add_argument('--enc_in', type=int, default=1, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels

## train
parser.add_argument('--loss', default="mse", type=str)
parser.add_argument('--optimizer', default="adam", type=str)
parser.add_argument('--learning_rate', default=0.0001, type=float)
parser.add_argument('--scheduler', default="none", type=str)
parser.add_argument('--batch_size', default=512, type=int)
parser.add_argument('--epochs', default=30, type=int)
# parser.add_argument('--validation_size', default=0.2, type=float)
parser.add_argument('--seed', default=826, type=int)
parser.add_argument('--mixed_precision', default=16, type=int)
parser.add_argument('--device', nargs='+', default=[0], type=int)
parser.add_argument('--num_workers', default=0, type=int)

args = parser.parse_args('')

wandb.config.update(args)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
# VALIDATION_SIZE = args.validation_size
SEED = args.seed

def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(SEED)

set_seeds()

submission_id = f"{parser.description}_{args.data}_pl{args.pred_len}"
submission_id

## Input/Dataset.py

In [ ]:
if args.data == "ETTh1":
    df = pd.read_csv("dataset/ETT-small/ETTh1.csv")
if args.data == "ETTh2":
    df = pd.read_csv("dataset/ETT-small/ETTh2.csv")
if args.data == "ETTm1":
    df = pd.read_csv("dataset/ETT-small/ETTm1.csv")
if args.data == "ETTm2":
    df = pd.read_csv("dataset/ETT-small/ETTm2.csv")
if args.data == "Exchange":
    df = pd.read_csv("dataset/exchange_rate/exchange_rate.csv")

# data = data.drop("date", axis=1)
df = df[["OT"]]

df.head()

In [ ]:
class Custom_Seq_Dataset(Dataset):
    def __init__(self, data, input_len,label_len, pred_len, target):
        self.data = data
        self.target = data[target].values #numpy
        self.data = data.values
        self.input_len = input_len
        self.label_len = label_len
        self.pred_len = pred_len

        
    def __len__(self):
        return len(self.data) - self.input_len - self.label_len + 1

    def __getitem__(self, idx):
        # feature and target
        feature = self.data[idx:idx+self.input_len]
        target = self.target[idx+(self.input_len-1):(idx+self.input_len + self.label_len-1)]

        # convert to tensor and reshape
        feature = torch.tensor(feature, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32).unsqueeze(1) #unsqueeze, 차원확장 (feature 차원 추가)
        
        return feature, target
      
data = Custom_Seq_Dataset(data=df, input_len=args.seq_len, label_len=args.label_len, pred_len=args.pred_len, target='OT') #Hyperparameter
    

#분할    
train_size = int(0.7 * len(data))  #분할했을 때 수 반환해서 미리 체크
val_size = int(0.1 * len(data))    
test_size = len(data) - train_size - val_size

train_indices = list(range(train_size)) #데이터를 시간의 순서대로 분할하기 위해 순서대로 인덱스 생성
val_indices = list(range(train_size, train_size + val_size))
test_indices = list(range(train_size + val_size, len(data)))

from torch.utils.data import Subset #하위 데이터로 나누기.
train_set = Subset(data, train_indices) #(데이터 셋, 인덱스).
val_set = Subset(data, val_indices)
test_set = Subset(data, test_indices)

## Input/DataLoader.py

In [ ]:
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)
val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

train_features, train_labels = next(iter(train_loader))
test_features, test_labels = next(iter(test_loader))

#check it
print(train_labels.shape)
print(train_features.shape)
print(test_labels.shape)
print(test_features.shape)

## Model/GD-MLP.py

In [ ]:
# ([batch_size, seq_len, features)]
# ex) batch_size = 8, seq_len = 336, features = 1, hidden_uits = 512
# input.shape = torch.Size([8, 336, 1)]

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    * decomposition Source from autoformer, Dlinear
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1) # -> torch.Size([8, 360, 1])
        x = self.avg(x.permute(0, 2, 1)) # -> torch.Size([8, 1, 336])
        x = x.permute(0, 2, 1) # -> torch.Size([8, 336, 1])
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    * decomposition Source from autoformer, Dlinear
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        trend = self.moving_avg(x)
        res = x - trend
        return res, trend #torch.Size([8, 336, 1])


class gated_mlp (nn.Module):

    """
    MLP block(gated_mlp)
    """
    def __init__(self, seq_len, num_features, pred_len, hidden_units):
        super(gated_mlp, self).__init__()
        self.seq_len = seq_len #인풋 길이
        self.num_features = num_features #피처 수
        self.pred_len = pred_len # 예측 길이
        self.hidden_units = hidden_units #노드 수
        kernel_size = 25 #same as Autoformer, Dlinear
        self.decomposition = series_decomp(kernel_size)
        
        # self.input_layer = nn.Linear(self.num_features, 1)


        reduction = 4
        
        self.input_gate1 = nn.Sequential(
            nn.Linear(self.seq_len, self.seq_len//reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(self.seq_len//reduction, self.seq_len, bias=False),
            nn.Sigmoid()
        ) #Input_gate
        
        self.input_gate2 = nn.Sequential(
            nn.Linear(self.seq_len, self.seq_len//reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(self.seq_len//reduction, self.seq_len, bias=False),
            nn.Sigmoid()
        ) #Input_gate


        # self.trend_mlp = nn.Sequential(
        #     nn.Linear(self.seq_len, self.hidden_units),
        #     nn.Linear(self.hidden_units, self.hidden_units)
        # ) # MLP for Trend


        # self.residual_mlp = nn.Sequential(
        #     nn.Linear(self.seq_len, self.hidden_units),
        #     nn.Linear(self.hidden_units, self.hidden_units)
        # ) # MLP for Residual



    def forward(self, x):
        
        residual_train,trend_train = self.decomposition(x) # torch.Size([8, 336, 1])

        #input_layer #not necessary
        # trend_train = self.input_layer(trend_train) #-> torch.Size([8, 336, 1])
        # residual_train = self.input_layer(residual_train)

        residual_train, trend_train = residual_train.permute(0,2,1), trend_train.permute(0,2,1) #-> torch.Size([8, 1, 336])
        
        # input_gate
        # i_gate_t = self.input_gate1(trend_train) #-> torch.Size([8, 1, 336])
        # trend_train = trend_train * i_gate_t #-> torch.Size([8, 1, 336])

        i_gate_r = self.input_gate2(residual_train) #-> torch.Size([8, 1, 336])
        residual_train = residual_train * i_gate_r #-> torch.Size([8, 1, 336])

        # trend MLP (gated)
        # trend_mlp = self.trend_mlp(trend_train) #MLP 통과 #-> torch.Size([8, 1, 512])

        # # residual MLP(gated)
        # residual_mlp = self.residual_mlp(residual_train) #MLP 통과 #-> torch.Size([8, 1, 512])
      

        # return trend_mlp, residual_mlp #torch.Size([8, 1, 512])
        return trend_train, residual_train #torch.Size([8, 1, 512])



class gated_sum(nn.Module):

    """
    Composing block with gate
    """
     
    def __init__(self, seq_len, num_features, pred_len, hidden_units):
        super(gated_sum, self).__init__()
        self.seq_len = seq_len #인풋 길이
        self.num_features = num_features #same as gated mlp
        self.pred_len = pred_len
        self.hidden_units = hidden_units
        kernel_size = 25
        self.decomposition = series_decomp(kernel_size)
        self.gated_mlp = gated_mlp(seq_len, num_features, pred_len, hidden_units) 
        
        self.output_layer1 = nn.Linear(self.seq_len, self.pred_len) #pred_len으로 선형변환하기 위한 output_layer
        self.output_layer2 = nn.Linear(self.seq_len, self.pred_len) 
        
        #Output gate
        self.output_gate = nn.Sequential(
            nn.Linear(self.pred_len, 1), #가중합 하기 위한 gate  
            nn.Sigmoid()
        )

    
    def forward(self, x):
        
        # gated_mlp
        trend_mlp, residual_mlp = self.gated_mlp(x)  #gated_mlp 블록을 통과 #->torch.Size([8, 1, 512])

        # output layer
        output_trend = self.output_layer1(trend_mlp)  #output_layer 통과 #->torch.Size([8, 1, 96])
        output_residual = self.output_layer2(residual_mlp) #output_layer 통과 #->torch.Size([8, 1, 96])
        
        # combine trend and residual MLPs with weighted sum
        trend_weight = self.output_gate(output_trend) # gate 통과 #->torch.Size([8, 1, 1])
        residual_weight = (1-trend_weight)

        #trend_weight,residual_weight = trend_weight.permute(0,2,1), residual_weight.permute(0,2,1)

        weighted_sum = (output_trend * trend_weight) + (output_residual * residual_weight) # Weighted sum == Final Output #->torch.Size([8, 1, 96])

        # weighted_sum = output_trend + output_residual
        
        return weighted_sum.permute(0,2,1) #Final Output #->torch.Size([8, 96, 1])

## DLinear.py

In [ ]:
# class moving_avg(nn.Module):
#     """
#     Moving average block to highlight the trend of time series
#     """
#     def __init__(self, kernel_size, stride):
#         super(moving_avg, self).__init__()
#         self.kernel_size = kernel_size
#         self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

#     def forward(self, x):
#         # padding on the both ends of time series
#         front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
#         end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
#         x = torch.cat([front, x, end], dim=1)
#         x = self.avg(x.permute(0, 2, 1))
#         x = x.permute(0, 2, 1)
#         return x


# class series_decomp(nn.Module):
#     """
#     Series decomposition block
#     """
#     def __init__(self, kernel_size):
#         super(series_decomp, self).__init__()
#         self.moving_avg = moving_avg(kernel_size, stride=1)

#     def forward(self, x):
#         moving_mean = self.moving_avg(x)
#         res = x - moving_mean
#         return res, moving_mean

# class Model(nn.Module):
#     """
#     Decomposition-Linear
#     """
#     def __init__(self, configs):
#         super(Model, self).__init__()
#         self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len

#         # Decompsition Kernel Size
#         kernel_size = 25
#         self.decompsition = series_decomp(kernel_size)
#         self.individual = configs.individual
#         self.channels = configs.enc_in

#         if self.individual:
#             self.Linear_Seasonal = nn.ModuleList()
#             self.Linear_Trend = nn.ModuleList()
            
#             for i in range(self.channels):
#                 self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
#                 self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

#                 # Use this two lines if you want to visualize the weights
#                 # self.Linear_Seasonal[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#                 # self.Linear_Trend[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#         else:
#             self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
#             self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
#             # Use this two lines if you want to visualize the weights
#             # self.Linear_Seasonal.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#             # self.Linear_Trend.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

#     def forward(self, x):
#         # x: [Batch, Input length, Channel]
#         seasonal_init, trend_init = self.decompsition(x)
#         seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
#         if self.individual:
#             seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
#             trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
#             for i in range(self.channels):
#                 seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
#                 trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
#         else:
#             seasonal_output = self.Linear_Seasonal(seasonal_init)
#             trend_output = self.Linear_Trend(trend_init)

#         x = seasonal_output + trend_output
#         return x.permute(0,2,1) # to [Batch, Output length, Channel]

## train.py

In [ ]:
class GDMLP(pl.LightningModule):
    def __init__(self, backbone, args):
        super().__init__()
        self.backbone = backbone

    def forward(self, x):
        return self.backbone(x)

    def step(self, batch):
        x = batch[0]
        y = batch[1]
        y_hat = self.forward(x)
        loss = nn.MSELoss()(y_hat, y)
        return loss, y, y_hat

    def metric(self, pred, true):
        mae = np.mean(np.abs(pred - true))
        mse = np.mean((pred - true) ** 2)
        return mae, mse

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        pred = y_hat.detach().cpu().numpy()
        true = y.detach().cpu().numpy()
        mae, mse = self.metric(pred, true)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_mae", mae, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_mse", mse, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        pred = y_hat.detach().cpu().numpy()
        true = y.detach().cpu().numpy()
        mae, mse = self.metric(pred, true)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_mae", mae, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_mse", mse, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        pred = y_hat.detach().cpu().numpy()
        true = y.detach().cpu().numpy()
        mae, mse = self.metric(pred, true)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_mae", mae, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_mse", mse, on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x = batch[0]
        y_hat = self.forward(x)
        return y_hat

    def configure_optimizers(self):
        if args.optimizer == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=args.learning_rate, momentum=0.9)
        if args.optimizer == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=args.learning_rate)
        if args.optimizer == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=args.learning_rate)
        
        if args.scheduler == "none":
            return optimizer
        if args.scheduler == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer=optimizer,
                step_size=2,
                gamma=0.9,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer=optimizer,
                T_max=EPOCHS,
                eta_min=1e-5,
            )
        if args.scheduler == "onecyclelr":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer=optimizer,
                max_lr=args.learning_rate,
                epochs=EPOCHS,
                steps_per_epoch=int(len(train_index) / BATCH_SIZE),
                pct_start=0.1,
            )
            return [optimizer], [scheduler]

## main.py

In [ ]:
model = GDMLP(gated_sum(args.seq_len, args.num_features, args.pred_len, args.hidden_units), args)

callbacks = [
    pl.callbacks.ModelCheckpoint(
        dirpath="saved/", filename=f"{submission_id}",
        monitor="val_loss", mode="min",
    ),
]

trainer = pl.Trainer(
    max_epochs=EPOCHS, accelerator="auto", callbacks=callbacks,
    precision=args.mixed_precision, logger=wandb_logger,
    devices=args.device
)

trainer.fit(model, train_loader, val_loader)

ckpt = torch.load(f"saved/{submission_id}.ckpt", map_location=torch.device(device))
model.load_state_dict(ckpt['state_dict'])

## test.py

eval_dict = trainer.validate(model, dataloaders=val_loader)[0]
wandb.log({'eval_loss': eval_dict["val_loss"]})
wandb.log({'eval_mae': eval_dict["val_mae"]})
wandb.log({'eval_mse': eval_dict["val_mse"]})

y_preds = trainer.predict(model, dataloaders=test_loader)
pred = np.vstack(y_preds)

In [ ]:
true = []
for i in test_set:
    true.append(i[1])

true = np.stack(true)

mae = np.mean(np.abs(pred - true))
mse = np.mean((pred - true) ** 2)

print(f"test_mae: {mae}")
print(f"test_mse: {mse}")

wandb.log({'test_mae': mae})
wandb.log({'test_mse': mse})

In [ ]:
wandb.finish()